In [127]:
import pandas as pd
import re
from dateutil.relativedelta import relativedelta


In [36]:
df = pd.read_csv("data/raw/McDonald_s_Reviews.csv", encoding='latin-1')


# Découverte du Dataset

In [37]:
df.columns

Index(['reviewer_id', 'store_name', 'category', 'store_address', 'latitude ',
       'longitude', 'rating_count', 'review_time', 'review', 'rating'],
      dtype='object')

In [38]:
df.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


In [39]:
df.describe(include="all")

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
count,33396.000000,33396,33396,33396,32736.000000,32736.000000,33396,33396,33396,33396
unique,NaN,2,1,40,NaN,NaN,51,39,22285,5
top,NaN,McDonald's,Fast food restaurant,"9814 International Dr, Orlando, FL 32819, Unit...",NaN,NaN,"2,810",4 years ago,Excellent,5 stars
freq,NaN,33325,33396,1890,NaN,NaN,1140,6740,2148,10274
mean,16698.500000,NaN,NaN,NaN,34.442546,-90.647033,NaN,NaN,NaN,NaN
std,9640.739131,NaN,NaN,NaN,5.344116,16.594844,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,25.790295,-121.995421,NaN,NaN,NaN,NaN
25%,8349.750000,NaN,NaN,NaN,28.655350,-97.792874,NaN,NaN,NaN,NaN
50%,16698.500000,NaN,NaN,NaN,33.931261,-81.471414,NaN,NaN,NaN,NaN
75%,25047.250000,NaN,NaN,NaN,40.727401,-75.399919,NaN,NaN,NaN,NaN


In [40]:
# compter les enventuels doublons en ignorant la colonne 'reviewer_id' et "rating_count"
len(df)-len(df.drop_duplicates(subset=df.columns.difference(['reviewer_id',"rating_count"])))


8484

In [41]:
df[df.isnull().any(axis = 1)].head(3)
#les valeurs longitudes et latitudes manquantes ont également des adresses incorrectes.

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
22141,22142,McDonald's,Fast food restaurant,2476 Kalï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿...,NaN,NaN,"2,175",3 months ago,Breakfast specials are good. The sausage burri...,4 stars
22142,22143,McDonald's,Fast food restaurant,2476 Kalï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿...,NaN,NaN,"2,175",a year ago,This isn't your typical McDonald's. This place...,5 stars
22143,22144,McDonald's,Fast food restaurant,2476 Kalï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿...,NaN,NaN,"2,175",2 weeks ago,This place was serving good quality breakfast ...,4 stars


In [42]:
df.store_name.value_counts()
#pas de distinction notable sur les les store_name mal écrits

store_name
McDonald's       33325
ýýýMcDonald's       71
Name: count, dtype: int64

In [43]:
df.review.sort_values(ascending=False) #des reviews avec des caractères illisibles

6163     ýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýý...
29362                    ýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýý
4898                  ýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýýý ýýý
15018                                ýýýýýýýýýýýýýýýýýýýýý
12379                             ýýýýýýýýýýýýýýýýýýýý ýýý
                               ...                        
21773                   "24 hour" store that is closed....
7110     !! Fraud location !! !!!Fraud location !!!, th...
700                  waiting time was longer than expected
661                  waiting time was longer than expected
656                          staff was rude and dismissive
Name: review, Length: 33396, dtype: object

# Cleaning

In [144]:
df1=df.copy()

In [145]:
# Correcting review_time
df1.loc[df1['review_time'].str.startswith('a'), 'review_time'] = \
    df1.loc[df1['review_time'].str.startswith('a'), 'review_time'].str.replace('a', '1', n=1)

In [146]:
# Correcting store_address
df1['store_address'] = df1['store_address'].str.replace('ï¿½', '')

df1.loc[df1['store_address'] == '2476 Kal', 'store_address'] = '2476 Kalakaua Ave, Honolulu, HI 96815, United States'

mask = df1['store_address'] == '2476 Kalakaua Ave, Honolulu, HI 96815, United States'
df1.loc[mask, ['latitude ', 'longitude']] = df1.loc[mask, ['latitude ', 'longitude']].fillna({
    'latitude ': 21.274506, 
    'longitude': -157.824215
})

In [147]:
# Correcting latitude
df1.rename(columns={'latitude ': 'latitude'}, inplace=True)

## Feature engineering

### Feauture engineering on review_time column

In [148]:
# Create function to parse the column review_time with the current date
def parse_relative_date(s, current_date):

    parts = s.strip().split()
    if len(parts) != 3:
        return None

    # This checks if the number is represented as "a" or "an" , as shown the dataset
    # we have columns that says a monthh ago which equivalent to 1 month ago
    num_str = parts[0].lower()
    if num_str in ['a', 'an']:
        num = 1
    else:
        try:
            num = int(num_str)
        except ValueError:
            return None

    unit = parts[1].lower()

    # This adjusts the current date based on the chosen unit
    if 'day' in unit:
        abs_date = current_date - pd.Timedelta(days=num)
    elif 'month' in unit:
        abs_date = current_date - relativedelta(months=num)
    elif 'year' in unit:
        abs_date = current_date - relativedelta(years=num)
    else:
        return None

    # Normalize to remove the time (sets time to 00:00:00)
    return abs_date.normalize()

# Current date timestamp
current_date = pd.Timestamp.now()

# Apply the function to create a new column with the actual dates
df1['review_date'] = df1['review_time'].apply(lambda x: parse_relative_date(x, current_date))

# Drop the column df1['review_time']
df1 = df1.drop(columns='review_time')



### Feauture engineering on store_address column

In [149]:
df1[['City', 'State']] = df1['store_address'].apply(lambda x: pd.Series(x.split(', ')[-3:-1]))

In [150]:
df1["State"]=df1["State"].str.split().str[0]

In [151]:
df1["store_address"]=df1["store_address"].str.split(",").str[0]

In [152]:
df1=df1[['reviewer_id',
        'store_name','category','store_address', 'latitude', 'longitude','City', 'State',
        'rating_count','review','rating', 'review_date']]

### Feauture engineering on rating column

In [153]:
# Remove the 'star' in the column rating 
df1["rating"] = (df1["rating"].str.split(" ").str[0].astype(int))

### Drop unnecessary columns

In [154]:
# Drop columns, 'store_name', 'category','rating_count'
df1 = df1.drop(columns=['store_name', 'category','rating_count'])

## Check unique values and duplicates

In [155]:
# Check unique values
columns_to_check = ['reviewer_id', 'store_address', 'latitude', 'longitude', 'City', 'State','review',
       'rating', 'review_date' ]

unique_counts = df1[columns_to_check].nunique()

print(unique_counts)

reviewer_id      33396
store_address       40
latitude            40
longitude           40
City                27
State               12
review           22285
rating               5
review_date         29
dtype: int64


In [156]:
# Check for duplicates
duplicate_rows = df1[df1.duplicated()]
print(f"Total duplicate rows: {duplicate_rows.shape[0]}")

Total duplicate rows: 0


## Clean column 'review' and remove special characters

In [157]:
df2 = df1.copy()

In [158]:
# Create function to remove special characters in the review column
def clean_review(review):
    #review = review.lower()
    review = review.replace('½ï', '').replace('ý', '')
    review = re.sub(r'[^a-zA-Z\s]', '', review)
    review = re.sub(r'\s+', ' ', review).strip()
    return review

df2['clean_reviews'] = df2['review'].apply(clean_review)

print(df2[['clean_reviews']])

                                           clean_reviews
0      Why does it look like someone spit on my food ...
1      Itd McDonalds It is what it is as far as the f...
2      Made a mobile order got to the speaker and che...
3      My mc Crispy chicken sandwich was customer ser...
4      I repeat my order times in the drive thru and ...
...                                                  ...
33391                         They treated me very badly
33392                           The service is very good
33393                         To remove hunger is enough
33394   Its good but lately it has become very expensive
33395                          they took good care of me

[33396 rows x 1 columns]


In [159]:
# replace special characters in the review column
def review(review):
    review = review.replace('½ï', '.').replace('ý', '.').replace('ï','.').replace('¿','.')
    review = re.sub(r'\s+', ' ', review).strip()
    return review

df2['review'] = df2['review'].apply(review)

print(df2[['review']])

                                                  review
0      Why does it look like someone spit on my food?...
1      It'd McDonalds. It is what it is as far as the...
2      Made a mobile order got to the speaker and che...
3      My mc. Crispy chicken sandwich was ..............
4      I repeat my order 3 times in the drive thru, a...
...                                                  ...
33391                        They treated me very badly.
33392                           The service is very good
33393                         To remove hunger is enough
33394  It's good, but lately it has become very expen...
33395                          they took good care of me

[33396 rows x 1 columns]


## Create new column 'actual_sentiment' and assign sentiments(negative, positive and neutral) for each rating

In [160]:
# Function to add column for the actual review: positive : ratings > 3,
# neutral: ratings == 3, negative : ratings <3.
def actual_sentiment(data):
    if data['rating'] > 3:
        return "positive" 
    elif data["rating"] == 3:
        return "neutral"
    else:
        return "negative"

# Apply the function to each row
df2['actual_sentiment'] = df2.apply(actual_sentiment, axis=1)

## Check duplicates in the 'clean_review' column

In [161]:
duplicates = df2[df2['clean_reviews'].duplicated()]
print(f"Total duplicate reviews: {duplicates.shape[0]}")
duplicates.head()

Total duplicate reviews: 11581


,reviewer_id,store_address,latitude,longitude,City,State,review,rating,review_date,clean_reviews,actual_sentiment
426,427,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,Good food and good service,5,2021-04-14,Good food and good service,positive
463,464,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,Good,5,2024-04-14,Good,positive
470,471,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,Great service,4,2024-04-14,Great service,positive
479,480,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,Fast.,3,2020-04-14,Fast,neutral
495,496,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,Great service,4,2022-04-14,Great service,positive


In [162]:
# Drop duplicates in the column
df2 = df2.drop_duplicates(subset='clean_reviews')
print(f"New dataset size: {df2.shape[0]}")

New dataset size: 21815


 ## Fill Missing Values

In [163]:
df2.isnull().sum()

reviewer_id           0
store_address         0
latitude              0
longitude             0
City                  0
State                 0
review                0
rating                0
review_date         308
clean_reviews         0
actual_sentiment      0
dtype: int64

In [164]:
#Fill missing dates of today's date

# Get yesterday's date and normalize to remove the time component
yesterday = (pd.Timestamp('today') - pd.Timedelta(days=1)).normalize()

# Fill missing values in the 'review_date' column with yesterday's date
df2['review_date'] = df2['review_date'].fillna(yesterday)

In [165]:
df2.head()

,reviewer_id,store_address,latitude,longitude,City,State,review,rating,review_date,clean_reviews,actual_sentiment
0,1,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,Why does it look like someone spit on my food?...,1,2025-01-14,Why does it look like someone spit on my food ...,negative
1,2,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,It'd McDonalds. It is what it is as far as the...,4,2025-04-09,Itd McDonalds It is what it is as far as the f...,positive
2,3,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,Made a mobile order got to the speaker and che...,1,2025-04-09,Made a mobile order got to the speaker and che...,negative
3,4,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,My mc. Crispy chicken sandwich was ..............,5,2025-03-14,My mc Crispy chicken sandwich was customer ser...,positive
4,5,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,"I repeat my order 3 times in the drive thru, a...",1,2025-02-14,I repeat my order times in the drive thru and ...,negative


## Save clean Dataset

In [166]:
df2.to_csv('data/cleaned_data.csv',index=False)